In [1]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch 

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model(
    'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 1025, 1024) shaped tensor

output = model.forward_head(output, pre_logits=True)
# output is a (1, num_features) shaped tensor


top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

/mnt/hdd/eric/.conda/envs/mapv2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/hdd/eric/.conda/envs/mapv2/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [43]:
model

Eva(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (rope): RotaryEmbeddingCat()
  (blocks): ModuleList(
    (0-23): 24 x EvaBlock(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): EvaAttention(
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): SwiGLU(
        (fc1_g): Linear(in_features=1024, out_features=2730, bias=True)
        (fc1_x): Linear(in

In [34]:
data_config

{'input_size': (3, 448, 448),
 'interpolation': 'bicubic',
 'mean': (0.48145466, 0.4578275, 0.40821073),
 'std': (0.26862954, 0.26130258, 0.27577711),
 'crop_pct': 1.0,
 'crop_mode': 'squash'}

In [35]:
transforms

Compose(
    Resize(size=(448, 448), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(448, 448))
    ToTensor()
    Normalize(mean=tensor([0.4815, 0.4578, 0.4082]), std=tensor([0.2686, 0.2613, 0.2758]))
)

In [11]:
top5_probabilities.shape

torch.Size([1, 5])

In [12]:
top5_probabilities

tensor([[7.8750, 3.2418, 2.3577, 2.2745, 2.0706]], grad_fn=<TopkBackward0>)

In [13]:
eval("van")

NameError: name 'van' is not defined

In [36]:
import torch
import yaml
import argparse
import time
from pathlib import Path
from rich.console import Console
from rich.table import Table
from torchvision.datasets import *
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast
#import torchvision.transforms as transforms
import timm

#from datasets.transforms import get_train_transforms, get_val_transforms
#from models import *
#from utils.losses import LabelSmoothCrossEntropy, CrossEntropyLoss
#from utils.utils import fix_seeds, setup_cudnn, create_progress_bar
#from utils.metrics import compute_accuracy

from torchvision.datasets import ImageFolder
from typing import Optional, Callable
from pathlib import Path

import torch
import os 
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import cv2
import rasterio 
from urllib.request import urlopen


#-- Console 
console = Console()


#-- load for map your city 
#--- path
input_path = "/mnt/hdd/eric/.tmp_ipy/00.Data/Map_Your_City/building-age-dataset/"
train_path = input_path + "train/data/"
test_path = input_path + "test/data/"

#--- device
device = torch.device("cuda:0")
print('Device: ' + str(device))   

#--- Load csv files
train_df = pd.read_csv(input_path + "train/train-set.csv")
test_df = pd.read_csv(input_path + "test/test-set.csv") 

#--- data split 
names_data = os.listdir(train_path)
names_train, names_valid = train_test_split(names_data, test_size=0.1, random_state=1)

#--- dataset class 
class Map_Dataset(torch.utils.data.Dataset):      
    def __init__(self, list_IDs,transform): 
        self.list_IDs = list_IDs
        self.transform = transform
        
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, index): 
        ID = self.list_IDs[index] 
        X1 = Image.open(train_path + ID + '/street.jpg').convert('RGB')
        #X1 = cv2.imread(train_path + ID + '/street.jpg')
        #X1 = cv2.resize(X1, (256, 256))
        #X1 = np.asarray(X1)
        #X1 = np.transpose(X1,[1, 2, 0])
        X1 = self.transform(X1)
        #X1 = cv2.resize(X1, (256, 256)) 
        
        #X2 = cv2.imread(train_path + ID + '/orthophoto.tif')  
        #X2 = cv2.resize(X2, (256, 256)) 
        
        #X3 = rasterio.open(train_path + ID + '/s2_l2a.tif').read() 
        #X3 = np.transpose(X3, [1, 2, 0]) 
        
        y = int(open(train_path + ID + '/label.txt', "r").read())
        return X1, y 
    

Device: cuda:0


In [37]:
transforms

Compose(
    Resize(size=(448, 448), interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(448, 448))
    ToTensor()
    Normalize(mean=tensor([0.4815, 0.4578, 0.4082]), std=tensor([0.2686, 0.2613, 0.2758]))
)

In [38]:
map = Map_Dataset(names_train,transforms)

In [1]:
(2 / 1024) * 1e-3

1.953125e-06